In [4]:
import sys, os

In [5]:
import networkx as nx
from geopy.distance import great_circle
import geohash
import geopandas as gpd
import osmnx as ox
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from scipy.stats import truncnorm

import math
import contextily as cx
from shapely.geometry import Polygon,Point
from shapely import wkt,ops
import colorsys
from sklearn.preprocessing import StandardScaler
from scipy import spatial
import random
from datetime import datetime
import pointpats 
import pandana

In [6]:
import itertools

In [7]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

In [8]:
country_name = 'colombia'
country_abbrv = 'co'

In [ ]:
daily_od_fname = f'od_{country_abbrv}_agg5_3h.csv'
daily_od = pd.read_csv(daily_od_fname)

In [14]:
unique_times = list(np.unique([row.local_time[-19:] for i, row in daily_od.iterrows()]))
str_time_dict = {t:i for i, t in enumerate(unique_times)}
daily_od['time_id'] = daily_od.local_time.apply(lambda x: str_time_dict[x[-19:]])
daily_od['local_date'] = daily_od.local_date.astype(str).apply(lambda x: datetime.strptime(x, "%Y%m%d"))
daily_od['neighbours'] = daily_od['start_geohash5'].apply(lambda x: geohash.neighbors(x))
daily_od['wtd_length']= daily_od.m_length_m * daily_od.trip_count

In [10]:
geohashes = list(set(daily_od.start_geohash5.unique()).union(set(daily_od.end_geohash5.unique())))
geohash_coord = {gh:geohash.decode(gh) for gh in geohashes}
geohash_bbox = {gh: geohash.bbox(gh) for gh in geohashes}

In [11]:
bbox_gdf = gpd.GeoDataFrame([[gh, Polygon([(geohash_bbox[gh]['w'], geohash_bbox[gh]['s']),
                                     (geohash_bbox[gh]['e'], geohash_bbox[gh]['s']),
                                     (geohash_bbox[gh]['e'], geohash_bbox[gh]['n']), 
                                     (geohash_bbox[gh]['w'], geohash_bbox[gh]['n'])])] for gh in geohashes],
                       columns=['gh5', 'geometry'], geometry='geometry', crs="EPSG:4326")
bbox_dict = {gh:Polygon([(geohash_bbox[gh]['w'], geohash_bbox[gh]['s']),
                                     (geohash_bbox[gh]['e'], geohash_bbox[gh]['s']),
                                     (geohash_bbox[gh]['e'], geohash_bbox[gh]['n']), 
                                     (geohash_bbox[gh]['w'], geohash_bbox[gh]['n'])]) for gh in geohashes}

In [12]:
import reverse_geocode

def is_land(lat, lon):
    location = reverse_geocode.search([(lat, lon)])[0]
    return 'country' in location  # If the location contains a 'country', it's on land

# Get Road Network for Entire Country

In [ ]:
def get_chunks(bbox, chunk_size):
    min_lon, min_lat, max_lon, max_lat = bbox
    lon_steps = np.arange(min_lon, max_lon, chunk_size)
    lat_steps = np.arange(min_lat, max_lat, chunk_size)
    chunks = []
    for lon in lon_steps:
        for lat in lat_steps:
            chunks.append([lon, lat, lon + chunk_size, lat + chunk_size])
    return chunks

In [ ]:
country_bbox={
    'colombia':[-78.97,-4.22,-66.17,12.49],
    'mexico':[-118.6,14.39,-86.49,32.72],
    'india':[67.95,6.55,97.4,35.67],
}

country_projCRS={
    'mexico': 'epsg:6362',
    'colombia': 'epsg:9377',  
    'india': 'epsg:24378'     

}


In [ ]:
from pyrosm import OSM
bounds = country_bbox[country_name]  

osm = OSM(f'{country_name}_road_network.osm.pbf')
graph_nodes, graph_edges = osm.get_network(nodes=True, network_type='driving', bounding_box=bounds)

graph_nodes = graph_nodes.reset_index()
graph_edges = graph_edges.reset_index()

# Spatial join to assign gh5 to each node
graph_nodes = gpd.sjoin(graph_nodes, bbox_gdf[['gh5', 'geometry']], how='left', predicate='within')

gh5_osmid = {i: row.osmid for i, row in graph_nodes.groupby('gh5').agg({'osmid': lambda x: list(x)}).iterrows()}

graph_nodes = graph_nodes.set_index('osmid')

graph_edges = graph_edges[['u','v','key', 'geometry', 'length','maxspeed']]

graph_edges = graph_edges.drop_duplicates(subset=['u','v'])
graph_edges['u'] = graph_edges['u'].astype(int)
graph_edges['v'] = graph_edges['v'].astype(int)

graph_edges = graph_edges[(graph_edges['u'].isin(graph_nodes.index)) & (graph_edges['v'].isin(graph_nodes.index))]
graph_nodes = graph_nodes[(graph_nodes.index.isin(graph_edges.u)) | (graph_nodes.index.isin(graph_edges.v))]

In [30]:
graph_nodes

,y,x,street_count,highway,ref,geometry,index_right,gh5
osmid,,,,,,,,
5747784382,3.945557,-70.606245,3,NaN,NaN,POINT (-70.60624 3.94556),NaN,NaN
5747930240,3.628122,-70.322200,3,NaN,NaN,POINT (-70.32220 3.62812),NaN,NaN
5747932474,3.928625,-70.570909,3,NaN,NaN,POINT (-70.57091 3.92862),NaN,NaN
6140934888,3.625660,-70.324619,1,NaN,NaN,POINT (-70.32462 3.62566),NaN,NaN
6140934889,3.623476,-70.324124,3,NaN,NaN,POINT (-70.32412 3.62348),NaN,NaN
...,...,...,...,...,...,...,...,...
11968312895,1.802609,-78.793283,3,NaN,NaN,POINT (-78.79328 1.80261),208.0,d0rfr
11968312896,1.802755,-78.793375,1,NaN,NaN,POINT (-78.79338 1.80276),208.0,d0rfr
11968312953,1.802338,-78.793284,3,NaN,NaN,POINT (-78.79328 1.80234),208.0,d0rfr


In [36]:
pnda_nx = pandana.Network(graph_nodes.x, graph_nodes.y, graph_edges.u, graph_edges.v, graph_edges[['length']])

Generating contraction hierarchies with 110 threads.
Setting CH node vector of size 1445057
Setting CH edge vector of size 3654712
Range graph removed 3408412 edges of 7309424
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


## Estimated Routes

In [1]:

def get_estimated_routes(real_df, gh5_osmid, iterations=100, sample_street=True):
    
    graph_points_df = []
    for iter_i in range(iterations):
        print(iter_i, end='\r')
        for i,row in real_df.iterrows():
    
            ### SAMPLE STREET NX POINTS ###
            if (row.start_geohash5 in gh5_osmid.keys()) and (row.end_geohash5 in gh5_osmid.keys()):
                points_og = random.choices(gh5_osmid[row.start_geohash5], k=row.trip_count)
                points_dest = random.choices(gh5_osmid[row.end_geohash5], k=row.trip_count)
                for o,d in zip(points_og, points_dest):
                    graph_points_df += [[row.start_geohash5, row.end_geohash5, row.local_date, row.time_id, o,d, iter_i]]
        
    graph_points_df = pd.DataFrame(graph_points_df, columns=['start_geohash5', 'end_geohash5', 'local_date', 'time_id', 'origin', 'destination', 'iteration'])
    return graph_points_df

In [2]:
def get_route_efficiency(real_df, filter_times, filter_dates, gh5_osmid, iterations=100):
    estimated_df = []
    for ft, fd in list(itertools.product(filter_times,filter_dates)):
        print('Fetching estimates for', str(fd)[:10], 'at', ft )
        filtered_df = real_df[(real_df.local_date==str(fd)[:10]) & (real_df.time_id==int(ft))]
        estimated_df.append(get_estimated_routes(filtered_df, gh5_osmid, iterations=iterations))
    estimated_df = pd.concat(estimated_df)
    return estimated_df

In [15]:
date_weekday = {d:d.weekday() < 5 for d in daily_od.local_date.unique()}
weekday_dates = [k for k,v in date_weekday.items() if v]
weekend_dates = [k for k,v in date_weekday.items() if not v]

## Estimated Routes for Agg

In [40]:
agg_weekday_od = daily_od[daily_od.local_date.isin(weekday_dates)]
agg_weekday_od = agg_weekday_od.groupby(['start_geohash5','end_geohash5','time_id']).agg({'trip_count':'mean',
                                                                                          'm_length_m':'mean',
                                                                                          'sd_length_m':'mean'}).reset_index()
agg_weekday_od['trip_count'] = agg_weekday_od['trip_count'].round().astype(int)
agg_weekday_od['local_date']  = 'agg'
agg_weekday_od.head()

agg_weekend_od = daily_od[daily_od.local_date.isin(weekend_dates)]
agg_weekend_od = agg_weekend_od.groupby(['start_geohash5','end_geohash5','time_id']).agg({'trip_count':'mean',
                                                                                          'm_length_m':'mean',
                                                                                          'sd_length_m':'mean'}).reset_index()
agg_weekend_od['trip_count'] = agg_weekend_od['trip_count'].round().astype(int)
agg_weekend_od['local_date']  = 'agg'
agg_weekend_od.head()

,start_geohash5,end_geohash5,time_id,trip_count,m_length_m,sd_length_m,local_date
0,d0rfr,d0rfr,2,14,2585.309754,3152.957604,agg
1,d0rfr,d0rfr,3,17,1781.322595,3466.759242,agg
2,d0rfr,d0rfr,4,15,1756.129379,1961.445673,agg
3,d0rfr,d0rfr,5,16,4665.225628,8826.063073,agg
4,d0rfr,d0rfr,6,15,1466.432141,1820.371835,agg


In [41]:
agg_weekend_eff = get_route_efficiency(agg_weekend_od, list(str_time_dict.values()), 
                                       ['agg'], gh5_osmid)

agg_weekday_eff = get_route_efficiency(agg_weekday_od, list(str_time_dict.values()), 
                                       ['agg'], gh5_osmid)

Fetching estimates for agg at 0
Fetching estimates for agg at 1
Fetching estimates for agg at 2
Fetching estimates for agg at 3
Fetching estimates for agg at 4
Fetching estimates for agg at 5
Fetching estimates for agg at 6
Fetching estimates for agg at 7
Fetching estimates for agg at 0
Fetching estimates for agg at 1
Fetching estimates for agg at 2
Fetching estimates for agg at 3
Fetching estimates for agg at 4
Fetching estimates for agg at 5
Fetching estimates for agg at 6
Fetching estimates for agg at 7


In [42]:
agg_weekend_eff['sp_lengths'] = pnda_nx.shortest_path_lengths(agg_weekend_eff.origin, 
                                                              agg_weekend_eff.destination, imp_name='length')

agg_weekday_eff['sp_lengths'] = pnda_nx.shortest_path_lengths(agg_weekday_eff.origin, 
                                                              agg_weekday_eff.destination, imp_name='length')

In [ ]:
agg_weekend_eff.to_csv('estimated_routes/'+country_abbrv+'_agg_weekend_eff.csv')
agg_weekday_eff.to_csv('estimated_routes/'+country_abbrv+'_agg_weekday_eff.csv')

## Calculate Efficiency

In [44]:
def get_efficiency_random(real_data, estimated_data, urbanicity_df):
    local_dates = list(estimated_data.local_date.unique())
    local_times = list(estimated_data.time_id.unique())
    # print(local_dates, local_times)
    
    filtered_rd = real_data[(real_data.local_date.isin(local_dates)) & (real_data.time_id.isin(local_times))]
    R = filtered_rd.groupby(['start_geohash5','local_date','time_id']).agg({'trip_count':'sum', 
                                                                            'wtd_length':'sum'}).reset_index()
    R['avg_len_real'] = R['wtd_length']/R['trip_count']
    E = estimated_data.groupby(['start_geohash5','local_date', 
                                'time_id', 'iteration']).agg({'sp_lengths':'mean', 
                                                              'origin':'count'}).reset_index().rename({'origin':'trip_count'},
                                                                                                      axis=1) 
    E = E.groupby(['start_geohash5','local_date','time_id']).agg({'sp_lengths':'mean','trip_count':'mean'}).reset_index()

    delta_df = R[['start_geohash5','avg_len_real', 'time_id', 'local_date',
                  'trip_count']].rename({'start_geohash5':'gh5'},
                                        axis=1).merge(bbox_gdf, how='left', on='gh5')
    delta_df = E.drop('trip_count',axis=1).rename({'start_geohash5':'gh5', 
                                      'sp_lengths':'avg_len_est'}, 
                                                  axis=1).merge(delta_df, how='left', on=['gh5', 'local_date', 'time_id'])
    delta_df['delta'] = delta_df.avg_len_real - delta_df.avg_len_est 
    delta_df = delta_df.merge(urbanicity_df, how='left', on='gh5')
    return delta_df

In [45]:
def get_efficiency_agg(real_data, estimated_data, urbanicity_df, local_dates):
    # local_dates = list(estimated_data.local_date.unique())
    local_times = list(estimated_data.time_id.unique())
    print(local_dates, local_times)
    
    filtered_rd = real_data[(real_data.local_date.isin(local_dates)) & (real_data.time_id.isin(local_times))]
    R = filtered_rd.groupby(['start_geohash5', 'end_geohash5', 'time_id']).agg({'trip_count':'mean',
                                                                                'wtd_length':'mean'}).reset_index()
    
    R = R.groupby(['start_geohash5','time_id']).agg({'trip_count':'sum', 'wtd_length':'sum'}).reset_index()
    R['trip_count'] = R['trip_count'].round().astype(int)

    R['avg_len_real'] = R['wtd_length']/R['trip_count']
    E = estimated_data.groupby(['start_geohash5','end_geohash5',
                                'time_id']).agg({'sp_lengths':'mean', 
                                                              'origin':'count'}).reset_index().rename({'origin':'trip_count'},
                                                                                                      axis=1)
    E['wtd_length'] = E['sp_lengths']*E['trip_count']
    E = E.groupby(['start_geohash5','time_id']).agg({'wtd_length':'sum','trip_count':'sum'}).reset_index()
    E['avg_len_est'] = E['wtd_length']/E['trip_count']
    
    delta_df = R[['start_geohash5','avg_len_real', 'time_id', 
                  'trip_count']].rename({'start_geohash5':'gh5'},
                                        axis=1).merge(bbox_gdf, how='left', on='gh5')
    delta_df = E.drop(['trip_count','wtd_length'],axis=1).rename({'start_geohash5':'gh5'}, 
                                                  axis=1).merge(delta_df, how='left', on=['gh5', 'time_id'])
    delta_df['delta'] = delta_df.avg_len_real - delta_df.avg_len_est 
    delta_df = delta_df.merge(urbanicity_df, how='left', on='gh5')
    return delta_df

In [46]:
urbanicity_df = pd.read_csv(f'{country_abbrv}_urbanicity_v2.csv')

/home/jupyter-niyer/README


In [48]:
eff_agg_weekday = get_efficiency_agg(daily_od, agg_weekday_eff,urbanicity_df, weekday_dates)
eff_agg_weekend = get_efficiency_agg(daily_od, agg_weekend_eff,urbanicity_df, weekend_dates)

[Timestamp('2019-11-01 00:00:00'), Timestamp('2019-11-04 00:00:00'), Timestamp('2019-11-05 00:00:00'), Timestamp('2019-11-06 00:00:00'), Timestamp('2019-11-07 00:00:00'), Timestamp('2019-11-08 00:00:00'), Timestamp('2019-11-11 00:00:00'), Timestamp('2019-11-12 00:00:00'), Timestamp('2019-11-13 00:00:00'), Timestamp('2019-11-14 00:00:00'), Timestamp('2019-11-15 00:00:00'), Timestamp('2019-11-18 00:00:00'), Timestamp('2019-11-19 00:00:00'), Timestamp('2019-11-20 00:00:00'), Timestamp('2019-11-21 00:00:00'), Timestamp('2019-11-22 00:00:00'), Timestamp('2019-11-25 00:00:00'), Timestamp('2019-11-26 00:00:00'), Timestamp('2019-11-27 00:00:00'), Timestamp('2019-11-28 00:00:00'), Timestamp('2019-11-29 00:00:00'), Timestamp('2019-12-02 00:00:00'), Timestamp('2019-12-03 00:00:00'), Timestamp('2019-12-04 00:00:00'), Timestamp('2019-12-05 00:00:00'), Timestamp('2019-12-06 00:00:00'), Timestamp('2019-12-09 00:00:00'), Timestamp('2019-12-10 00:00:00'), Timestamp('2019-12-11 00:00:00'), Timestamp('20

In [ ]:
eff_agg_weekend.to_csv('estimated_routes/'+country_abbrv+'_agg_weekend_eff_final.csv')
eff_agg_weekday.to_csv('estimated_routes/'+country_abbrv+'_agg_weekday_eff_final.csv')
